In [10]:
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel

In [2]:
df1 = pd.read_excel('附件一：325个样本数据(处理后).xlsx', header=None)
df4 = pd.read_excel('附件四：354个操作变量信息.xlsx', header=None)

In [3]:
df1_2 = df1.drop(df1.index[0:1])
df1_2 = df1_2.drop(df1_2.columns[0:16], axis=1)
df1_2 = df1_2.drop(df1.index[2])

1、生成只包含操作变量的dataframe

In [4]:
array = np.array(df1_2)  # 将df转换为list对象
list = array.tolist()  # 将array转换为list
list = list[0]         # 获取第一行的数据
df1_2.columns = list
df1_2 = df1_2.drop(df1_2.index[0:1])
df1_2.index = map(str, np.arange(df1_2.index.size))

2、第一次缺失值处理

In [5]:
def judge1_1(df,t):
    x = df.index.size
    y = df.columns.size
    for i in range(y):
        a = 0
        b = 0
        for j in range(x):
            if(df.iat[j,i] == 0):
                a += 1
            else:
                b = b + df.iat[j,i]
        if(a > x/2):
            for j in range(x):
                df.iat[j,i] = 0
            t.append(i)
        else:
            for j in range(x):
                if(df.iat[j,i] == 0):
                    df.iat[j,i] = b/(x-a)
t1 = []
judge1_1(df1_2, t1)        

3、最大最小越限处理：根据附件4的最大最小值进行判断

In [6]:

df4_1 = pd.DataFrame(df4[3])
df4_1 = df4_1.drop(df4_1.index[0])
df4_1.columns = ["取值范围"]
df4_1.index = map(str, np.arange(354))

In [7]:
def judge2_1(df1, df2, t):
    a = 0
    for i in range(0, 354): 
        ul = re.sub('\(|\)','',df1.iat[i,0])
        ul = re.sub('\（|\）','',ul)
        if("--" in ul):
            upper = "-" + ul.rsplit("-",1)[1]   #上限
            lower = ul.rsplit("-",2)[0]         #下限
        else:
            upper = ul.rsplit("-",1)[1]         #上限
            lower = ul.rsplit("-",1)[0]         #下限
        upper = float(upper)
        lower = float(lower)
        for j in range(0, df2.index.size):
            if(float(df2.iat[j,i]) <= lower or float(df2.iat[j,i]) >= upper):
                df2.iat[j,i] = 0
                t.append(i)
t2 = []   
judge2_1(df4_1, df1_2, t2) 

4、异常值处理：按照3西格玛原则

In [8]:
def judge2_2(df3):
    a = np.std(df3, axis = 0, ddof =1)
    b = np.mean(df3)
    for i in range(0,354):
        for j in range(0,df3.index.size):
            if(np.abs(b[i]-df3.iat[j, i]) > 3*a[i]):
                df3.iat[j, i] = 0
judge2_2(df1_2)  

5、第二次缺失值处理

In [9]:
def judge1_1(df,t):
    x = df.index.size
    y = df.columns.size
    for i in range(y):
        a = 0
        b = 0
        for j in range(x):
            if(df.iat[j,i] == 0):
                a += 1
            else:
                b = b + df.iat[j,i]
        if(a > x/2):
            for j in range(x):
                df.iat[j,i] = 0
            t.append(i)
        else:
            for j in range(x):
                if(df.iat[j,i] == 0):
                    df.iat[j,i] = b/(x-a)
t1 = []
judge1_1(df1_2, t1)   

In [11]:
df1_2

,S-ZORB.CAL_H2.PV,S-ZORB.PDI_2102.PV,S-ZORB.PT_2801.PV,S-ZORB.FC_2801.PV,S-ZORB.TE_2103.PV,S-ZORB.TE_2005.PV,S-ZORB.PT_2101.PV,S-ZORB.PDT_2104.PV,S-ZORB.SIS_PDT_2103B.PV,S-ZORB.TC_2101.PV,...,S-ZORB.CAL_1.CANGLIANG.PV,S-ZORB.FT_1006.DACA.PV,S-ZORB.FT_5204.DACA.PV,S-ZORB.FT_1006.TOTALIZERA.PV,S-ZORB.FT_5204.TOTALIZERA.PV,S-ZORB.FT_1503.DACA.PV,S-ZORB.FT_1503.TOTALIZERA.PV,S-ZORB.FT_1504.DACA.PV,S-ZORB.FT_1504.TOTALIZERA.PV,S-ZORB.PC_1001A.PV
0,0.276318,31.211968,2.526857,647.75238,424.6159,421.608145,2.393991,101.29058,33.480159,240.088445,...,2.045436,6368.747,233.310805,83086802,832503.795,2216.40935,39063124.5,1840.1447,39608757,0.353271
1,0.277127,31.461969,2.527411,651.82192,420.953385,417.92783,2.394113,101.544925,33.324023,239.80982,...,2.025974,6360.6453,242.369205,82318954,803462.665,2370.5874,38810581.5,1641.7326,39389299,0.354504
2,0.277013,31.339998,2.528865,650.74715,423.941065,420.003515,2.394545,101.61832,33.251831,239.742695,...,1.964696,6504.9649,233.076925,82012004,791925.055,2326.46535,38693812,1600.67575,39312616.5,0.350181
3,0.277222,31.033443,2.530589,652.669055,423.722805,419.964775,2.392854,105.02976,33.197268,239.732465,...,2.039041,6506.82495,238.349935,81231373.5,762863.81,2495.2236,38410862.5,1563.71215,39120204.5,0.35393
4,0.27731,30.826894,2.530938,649.34092,427.132,422.128935,2.392715,105.937455,32.785944,239.835635,...,1.9869,6560.24225,236.57622,80915707.5,751362.3,2807.7891,38283000,1554.3574,39045953.5,0.358053
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
320,0.238174,18.169647,2.392515,819.708705,424.56721,423.932695,2.299256,52.717846,1293.641175,244.98636,...,2.237752,8328.97475,150.441617,84087479.25,2136687.475,9872.994216,1569503.75,2327.132132,1003276.5,-118.251152
321,0.239947,18.356451,2.393195,820.912883,425.062968,424.276302,2.298786,54.426241,1298.1621,244.93765,...,2.752566,8457.8608,148.748365,83687113.25,2129270.425,9872.994216,1521263.75,2327.132132,879436.8,-118.764337
322,0.237478,18.477196,2.398691,818.2461,425.3908,424.563592,2.301724,54.764742,1304.9434,245.025612,...,2.935758,8246.20925,150.095445,83087625.75,2118469.7,9872.994216,1448903.75,2327.132132,693676.8,-119.534108
323,0.235185,18.626541,2.394805,820.887623,426.057338,425.133608,2.296625,55.948335,1309.464275,244.818115,...,2.237752,7998.80235,149.533987,82696129.75,2110959.45,9872.994216,1400663.75,2327.132132,569836.8,-120.047287


In [89]:
df1_2.to_excel("df1_2.xlsx",  encoding= 'UTF-8')